In [1]:
from __future__ import absolute_import, division, print_function
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.signal import kaiser,kaiser_beta
import time
from scipy.stats import norm
import numpy as np
from scipy.stats import linregress
from lisaconstants import GM_SUN,c,ASTRONOMICAL_YEAR,ASTRONOMICAL_UNIT,SUN_SCHWARZSCHILD_RADIUS,OBLIQUITY

# Initial settings used in LISA instrument simulation for loading the correct position and velocity files

In [2]:
data_dir = './'

f_s = 4
sec = 1*3600



# Load the position and velocity data

In [3]:

positions_1 = np.genfromtxt(data_dir+'s_c_positions_{0}_Hz_{1}_sec_spacecraft_1.dat'.format(f_s,sec))
positions_2 = np.genfromtxt(data_dir+'s_c_positions_{0}_Hz_{1}_sec_spacecraft_2.dat'.format(f_s,sec))
positions_3 = np.genfromtxt(data_dir+'s_c_positions_{0}_Hz_{1}_sec_spacecraft_3.dat'.format(f_s,sec))

velocity_1 = np.genfromtxt(data_dir+'s_c_velocity_{0}_Hz_{1}_sec_spacecraft_1.dat'.format(f_s,sec))
velocity_2 = np.genfromtxt(data_dir+'s_c_velocity_{0}_Hz_{1}_sec_spacecraft_2.dat'.format(f_s,sec))
velocity_3 = np.genfromtxt(data_dir+'s_c_velocity_{0}_Hz_{1}_sec_spacecraft_3.dat'.format(f_s,sec))

# Magnitude of position and velocity vectors

In [4]:
mag_pos_1 = np.sqrt(positions_1.T[0]**2+positions_1.T[1]**2+positions_1.T[2]**2)     
mag_pos_2 = np.sqrt(positions_2.T[0]**2+positions_2.T[1]**2+positions_2.T[2]**2)     
mag_pos_3 = np.sqrt(positions_3.T[0]**2+positions_3.T[1]**2+positions_3.T[2]**2)     

mag_vel_1 = np.sqrt(velocity_1.T[0]**2+velocity_1.T[1]**2+velocity_1.T[2]**2)     
mag_vel_2 = np.sqrt(velocity_2.T[0]**2+velocity_2.T[1]**2+velocity_2.T[2]**2)     
mag_vel_3 = np.sqrt(velocity_3.T[0]**2+velocity_3.T[1]**2+velocity_3.T[2]**2)  

# Angular momentum vector and magnitude of each S/C

In [5]:
ang_momen_1 = np.cross(positions_1,velocity_1)
ang_momen_2 = np.cross(positions_2,velocity_2)
ang_momen_3 = np.cross(positions_3,velocity_3)

mag_ang_momen_1 = np.sqrt(ang_momen_1.T[0]**2+ang_momen_1.T[1]**2+ang_momen_1.T[2]**2) 
mag_ang_momen_2 = np.sqrt(ang_momen_2.T[0]**2+ang_momen_2.T[1]**2+ang_momen_2.T[2]**2) 
mag_ang_momen_3 = np.sqrt(ang_momen_3.T[0]**2+ang_momen_3.T[1]**2+ang_momen_3.T[2]**2) 

# Eccentricity vector and magnitude $e$ of each S/C

In [6]:
ecc_1 = np.cross(velocity_1,ang_momen_1).T/GM_SUN - positions_1.T/mag_pos_1
ecc_2 = np.cross(velocity_2,ang_momen_2).T/GM_SUN - positions_2.T/mag_pos_2
ecc_3 = np.cross(velocity_3,ang_momen_3).T/GM_SUN - positions_3.T/mag_pos_3

mag_ecc_1 = np.sqrt(ecc_1[0]**2+ecc_1[1]**2+ecc_1[2]**2)     
mag_ecc_2 = np.sqrt(ecc_2[0]**2+ecc_2[1]**2+ecc_2[2]**2)     
mag_ecc_3 = np.sqrt(ecc_3[0]**2+ecc_3[1]**2+ecc_3[2]**2)    

# Unit direction vector $\hat{n}$ and magnitude of each S/C

In [7]:
n_1 = np.array([-1*ang_momen_1.T[1],ang_momen_1.T[0],np.zeros(len(ang_momen_1.T[0]))])
n_2 = np.array([-1*ang_momen_2.T[1],ang_momen_2.T[0],np.zeros(len(ang_momen_2.T[0]))])
n_3 = np.array([-1*ang_momen_3.T[1],ang_momen_3.T[0],np.zeros(len(ang_momen_3.T[0]))])

mag_n_1 = np.sqrt(n_1[0]**2+n_1[1]**2+n_1[2]**2) 
mag_n_2 = np.sqrt(n_2[0]**2+n_2[1]**2+n_2[2]**2)     
mag_n_3 = np.sqrt(n_3[0]**2+n_3[1]**2+n_3[2]**2) 

# True anomaly $\nu(t)$ of each S/C

In [8]:

if all(np.sum(positions_1*velocity_1,axis=1)>=0.0):
    true_anomaly_1 = np.arccos(np.sum(ecc_1*positions_1.T,axis=0)/(mag_ecc_1*mag_pos_1))
else:
    true_anomaly_1 = 2.0*np.pi - np.arccos(np.sum(ecc_1*positions_1.T,axis=0)/(mag_ecc_1*mag_pos_1))

if all(np.sum(positions_2*velocity_2,axis=1)>=0.0):
    true_anomaly_2 = np.arccos(np.sum(ecc_2*positions_2.T,axis=0)/(mag_ecc_2*mag_pos_2))
else:
    true_anomaly_2 = 2.0*np.pi - np.arccos(np.sum(ecc_2*positions_2.T,axis=0)/(mag_ecc_2*mag_pos_2))	

if all(np.sum(positions_3*velocity_3,axis=1)>=0.0):
    true_anomaly_3 = np.arccos(np.sum(ecc_3*positions_3.T,axis=0)/(mag_ecc_3*mag_pos_3))
else:
    true_anomaly_3 = 2.0*np.pi - np.arccos(np.sum(ecc_3*positions_3.T,axis=0)/(mag_ecc_3*mag_pos_3))

# Inclination $\iota(t)$ of each S/C

In [9]:
inclination_1 = np.arccos(ang_momen_1.T[2]/mag_ang_momen_1)
inclination_2 = np.arccos(ang_momen_2.T[2]/mag_ang_momen_2)
inclination_3 = np.arccos(ang_momen_3.T[2]/mag_ang_momen_3)

# Eccentric anomaly $E(t)$ of each S/C

In [10]:
ecc_anomaly_1 = 2.0*np.arctan(np.tan(true_anomaly_1/2.0)/(np.sqrt((1.0+mag_ecc_1)/(1.0-mag_ecc_1))))
ecc_anomaly_2 = 2.0*np.arctan(np.tan(true_anomaly_2/2.0)/(np.sqrt((1.0+mag_ecc_2)/(1.0-mag_ecc_2))))
ecc_anomaly_3 = 2.0*np.arctan(np.tan(true_anomaly_3/2.0)/(np.sqrt((1.0+mag_ecc_3)/(1.0-mag_ecc_3))))

# RAAN $\Omega(t)$ of each S/C.

In [11]:
if all(n_1[1] >=0.0):
    raan_1 = np.arccos(n_1[0]/mag_n_1)
else:
    raan_1 = 2.0*np.pi - np.arccos(n_1[0]/mag_n_1)

if all(n_2[1] >=0.0):
    raan_2 = np.arccos(n_2[0]/mag_n_2)
else:
    raan_2 = 2.0*np.pi - np.arccos(n_2[0]/mag_n_2)
    
if all(n_3[1] >=0.0):
    raan_3 = np.arccos(n_3[0]/mag_n_3)
else:
    raan_3 = 2.0*np.pi - np.arccos(n_3[0]/mag_n_3)


# Arg. of perihelion $\omega(t)$ of each S/C.

In [12]:
if all(ecc_1[2]>=0.0):
    arg_per_1 = np.arccos(np.sum(n_1*ecc_1,axis=0)/(mag_n_1*mag_ecc_1))
else:
    arg_per_1 = 2.0*np.pi - np.arccos(np.sum(n_1*ecc_1,axis=0)/(mag_n_1*mag_ecc_1))
    
if all(ecc_2[2]>=0.0):
    arg_per_2 = np.arccos(np.sum(n_2*ecc_2,axis=0)/(mag_n_2*mag_ecc_2))
else:
    arg_per_2 = 2.0*np.pi - np.arccos(np.sum(n_2*ecc_2,axis=0)/(mag_n_2*mag_ecc_2))
    
if all(ecc_3[2]>=0.0):
    arg_per_3 = np.arccos(np.sum(n_3*ecc_3,axis=0)/(mag_n_3*mag_ecc_3))
else:

    arg_per_3 = 2.0*np.pi - np.arccos(np.sum(n_3*ecc_3,axis=0)/(mag_n_3*mag_ecc_3))

# Mean anomaly $M(t)$ of each S/C

In [13]:
mean_anomaly_1 = ecc_anomaly_1 - mag_ecc_1*np.sin(ecc_anomaly_1)
mean_anomaly_2 = ecc_anomaly_2 - mag_ecc_2*np.sin(ecc_anomaly_2)
mean_anomaly_3 = ecc_anomaly_3 - mag_ecc_3*np.sin(ecc_anomaly_3)

# Semi-major axis $a(t)$ of each S/C

In [14]:
semi_major_1 = 1.0/(2.0/mag_pos_1 - mag_vel_1**2/GM_SUN)
semi_major_2 = 1.0/(2.0/mag_pos_2 - mag_vel_2**2/GM_SUN)
semi_major_3 = 1.0/(2.0/mag_pos_3 - mag_vel_3**2/GM_SUN)

# Save the first value in time (at cut_off) for each Keplerian orbital quantity to file.

In [15]:
cut_off = 0
header = 'semi_major_1 semi_major_2 semi_major_3 mag_ecc_1 mag_ecc_2 mag_ecc_3 inclination_1 inclination_2 inclination_3 mean_anomaly_1 mean_anomaly_2 mean_anomaly_3 raan_1 raan_2 raan_3 arg_per_1 arg_per_2 arg_per_3'
elements = np.array([semi_major_1[cut_off],semi_major_2[cut_off],semi_major_3[cut_off],mag_ecc_1[cut_off],mag_ecc_2[cut_off],mag_ecc_3[cut_off],inclination_1[cut_off],inclination_2[cut_off],inclination_3[cut_off],mean_anomaly_1[cut_off],mean_anomaly_2[cut_off],mean_anomaly_3[cut_off],raan_1[cut_off],raan_2[cut_off],raan_3[cut_off],arg_per_1[cut_off],arg_per_2[cut_off],arg_per_3[cut_off]])
np.savetxt('elements_from_Cartesian_{0}_Hz_{1}_sec.dat'.format(f_s,sec),elements.T,header = header)